In [1]:
import asyncio
import difflib
import json
import sys
from functools import wraps

import pandas as pd
import plotly.io
from litellm import acompletion, completion_cost
from pydantic import BaseModel
from tqdm.asyncio import tqdm
from unidecode import unidecode

sys.path.append("../..")

plotly.io.renderers.default = "png"

## Make predictions for Sud Radio & CNEWS for one month


In [2]:
# Load data
df = pd.read_parquet("../../data/raw/18_channels_2023_09_to_2024_09.parquet")
df = df[df["channel_name"].isin(["sud-radio", "itele"]) & (df["start"] >= "2024-07-01")]

# Fixing some syntax issues in the data
# Mediatree issue with "' "
df["text"] = df["text"].str.replace("' ", "'")

df

,start,text,channel_name,channel_is_radio,channel_program_type,channel_program,themes,keywords,num_keywords,num_tokens
id,,,,,,,,,,
c945dffa14ec20dd14b3e6e1304fbb4b25bddae8895e50d72d42e049e0ecee2e,2024-07-01 06:22:00,à la gestion de votre bien en location de la r...,itele,False,Information en continu,Information en continu,"[""changement_climatique_causes"", ""changement_c...","[{""keyword"": ""m\u00e9thane"", ""timestamp"": 1719...",1,506
c90b7ed1e7139b209c7cd9ca6006f0fbfaf2de457c4f919aba6f4104056c5832,2024-07-01 07:12:00,xxl maison mobilier design et décoration de vo...,itele,False,Information en continu,Information en continu,"[""attenuation_climatique_solutions"", ""changeme...","[{""keyword"": ""transition \u00e9nerg\u00e9tique...",1,401
d206d8c5c48efe0a621275c4bbe627c72b4de484339aa75d39cfb5a82aa7052f,2024-07-01 07:50:00,répercutée sur les prix la hausse des coûts de...,itele,False,Information en continu,Information en continu,"[""ressources_solutions"", ""attenuation_climatiq...","[{""keyword"": ""biogaz"", ""timestamp"": 1719813025...",2,556
a8b1d5a95027030b142487be65621d16d928265ac18429a9ee356ccca2179ddf,2024-07-01 10:46:00,monde le prix du gaz en prenant en compte qu'i...,itele,False,Information en continu,Information en continu,"[""ressources_solutions"", ""attenuation_climatiq...","[{""keyword"": ""biogaz"", ""timestamp"": 1719823566...",1,498
f055f93229e59c00a3872625b0c4e557938279b71944983ada8e8c0f8fca7a77,2024-07-01 11:44:00,est encore acté avec cette image que je trouve...,itele,False,Information en continu,Information en continu,"[""changement_climatique_constat"", ""changement_...","[{""keyword"": ""eaux"", ""timestamp"": 171982713101...",2,596
...,...,...,...,...,...,...,...,...,...,...
121c2c2780f373e0d03ad266a5b4c11e87873e353954051693a978db123e1a04,2024-09-12 07:06:00,est la période la plus à risque pour les inond...,sud-radio,True,Information - Magazine,Le Grand Matin,"[""adaptation_climatique_solutions"", ""changemen...","[{""keyword"": ""inondation"", ""timestamp"": 172611...",2,575
10e4d16cad946340c67738bdaa487603d4635c6d2db978b63aff13916a49cfab,2024-09-12 08:00:00,solaire point fr votre expert en panneaux sola...,sud-radio,True,Information - Magazine,Le Grand Matin,"[""ressources_solutions"", ""changement_climatiqu...","[{""keyword"": ""panneaux solaires"", ""timestamp"":...",1,526
74f1b824b772877e6e05908a308a8ae31146b652ff50793bbd7fc9d4deb5e29c,2024-09-12 09:00:00,foyer trois mille neuf cent quatre-vingt-dix e...,sud-radio,True,Information - Magazine,Le Grand Matin,"[""ressources_solutions"", ""changement_climatiqu...","[{""keyword"": ""panneaux solaires"", ""timestamp"":...",1,520


In [3]:
# Limit concurrent requests to avoid API rate limiting
# (it depends on the model you use and your API tier)
semaphore = asyncio.Semaphore(20)


# Decorator that ensures `acompletion` uses the semaphore
def with_semaphore(acquire_semaphore):
    def decorator(func):
        @wraps(func)
        async def wrapper(*args, **kwargs):
            async with acquire_semaphore:
                return await func(*args, **kwargs)

        return wrapper

    return decorator


acompletion = with_semaphore(semaphore)(acompletion)


class MediatreePrediction(BaseModel):
    label_pred: str
    prompt_tokens: int
    completion_tokens: int
    total_tokens: int
    cost: float
    claim_pred: str | None = None


async def predict_cards_label(text: str) -> MediatreePrediction:
    system_prompt = """Tu es expert en désinformation sur les sujets environnementaux, expert en science climatique et sachant tout sur le GIEC. Je vais te donner un extrait d'une retranscription de 2 minutes d'un flux TV ou Radio. Parmis certains extraits, nous avons inséré un passage climatosceptique qui doit être factchecker. Ta tâche est de trouver ce passage climatosceptique (si il existe). Isole le passage à factchecker, puis fournis le label de la catégorie associé à ce passage. Ne t'intéresse qu'aux passages qui sont clairement climatosceptiques et ignore les thématiques sociales et/ou économiques. 

<catégories prédéfinies>
- 0_accepted: Lorsque l'ensemble de l'extrait est neutre ou reflète le consensus scientifique sur les questions climatiques, sans contenir de désinformation. Cette catégorie est aussi adaptée si l'extrait n'aborde pas le sujet du climat.
- 1_its_not_happening: Le réchauffement climatique n'a pas lieu.
- 2_its_not_us: Les humains ne sont pas à l'origine du réchauffement climatique.
- 3_its_not_bad: Les impacts climatiques ne sont pas mauvais.
- 4_solutions_wont_work: Les solutions climatiques ne fonctionneront pas.
- 5_science_is_unreliable: Les mouvements pour le climat et la science du climat ne sont pas fiables.
</catégories prédéfinies>

<exemples>

<exemple_0>
<extrait>
les scientifiques observent une hausse des températures mondiales, entraînant des changements climatiques visibles ils soulignent la nécessité d'agir pour en limiter les impacts [...]
</extrait>
<passage climatosceptique>
AUCUN
</passage climatosceptique>
<catégorie>0_accepted</catégorie>
</exemple_0>

<exemple_1>
<extrait>
[...]
</extrait>
<passage climatosceptique>
la fréquence des discussions sur la crise climatique semble augmenter mais certains experts proposent que les avertissements sur le climat pourraient être exagérés faisant souvent valoir que les fluctuations climatiques sont tout à fait naturelles et que les vérités scientifiques sont parfois altérées pour servir des intérêts politiques
</passage climatosceptique>
<catégorie>1_its_not_happening</catégorie>
</exemple_1>

<exemple_2>
<extrait>
[...]
</extrait>
<passage climatosceptique>
il est contesté que la contribution humaine aux niveaux de CO2 soit réellement significative certains experts soutiennent que les océans jouent un rôle bien plus important dans l'émission de ce gaz
</passage climatosceptique>
<catégorie>2_its_not_us</catégorie>
</exemple_2>

<exemple_3>
<extrait>
[...]
</extrait>
<passage climatosceptique>
il est important de noter que beaucoup de régions touchées par des événements climatiques extrêmes retrouvent leur équilibre naturel et peuvent même bénéficier de ces changements car plusieurs études montrent que certaines espèces et écosystèmes s'adaptent efficacement aux variations climatiques
</passage climatosceptique>
<catégorie>3_its_not_bad</catégorie>
</exemple_3>

<exemple_4>
<extrait>
[...]
</extrait>
<passage climatosceptique>
de nombreux experts remettent en question l'efficacité des initiatives visant à réduire la pollution, arguant qu'elles ne font qu'ajouter des coûts économiques sans réel impact positif sur l'environnement
</passage climatosceptique>
<catégorie>4_solutions_wont_work</catégorie>
</exemple_4>

<exemple_5>
<extrait>
[...]
</extrait>
<passage climatosceptique>
En réfléchissant à la situation on peut voir un parallèle avec le débat climatique de nombreux sceptiques pensent que le discours autour du climat est devenu une sorte de dogme où la critique est difficilement tolérée
</passage climatosceptique>
<catégorie>5_science_is_unreliable</catégorie>
</exemple_5>

</exemples>
"""
    response = await acompletion(
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": f"<extrait>\n{text.strip()}\n</extrait>"},
            {
                "role": "assistant",
                "content": "<passage climatosceptique>",
                "prefix": True,
            },
        ],
        model="anthropic.claude-3-haiku-20240307-v1:0",
        max_tokens=1000,
        temperature=0,
        stop=["</catégorie>"],
    )

    # Parse the response
    response_content: str = response.choices[0].message.content
    claim = response_content.split("</passage climatosceptique>")[0].strip()
    label_pred = response_content.split("<catégorie>")[1].strip()
    return MediatreePrediction(
        label_pred=label_pred,
        prompt_tokens=response.usage.prompt_tokens,
        completion_tokens=response.usage.completion_tokens,
        total_tokens=response.usage.total_tokens,
        cost=completion_cost(response),
        claim_pred=claim if label_pred != "0_accepted" else None,
    )

In [4]:
# Make predictions
mediatree_predictions: list[MediatreePrediction] = await tqdm.gather(
    *[predict_cards_label(text) for text in df["text"]]
)
mediatree_predictions_df = pd.DataFrame(
    [pred.model_dump(exclude_none=True) for pred in mediatree_predictions],
    index=df.index,
)
df = pd.concat([df, mediatree_predictions_df], axis=1)

100%|██████████| 2734/2734 [06:26<00:00,  7.07it/s]


In [6]:
# Save the result
df.to_csv(
    "../../data/experiments/label_studio_review/sud_radio_itele_since_2024_07.csv"
)

In [7]:
def show_llm_usage(df: pd.DataFrame) -> None:
    print("\nLLM USAGE\n=========\n")
    print(
        f"Median token usage:\n- Prompt: {int(df["prompt_tokens"].median())}\n"
        f"- Completion: {int(df["completion_tokens"].median())}\n"
        f"- Total: {int(df["total_tokens"].median())}"
    )
    print(f"\nTotal cost: ${df["cost"].sum():.3f}\n")


show_llm_usage(df)


LLM USAGE

Median token usage:
- Prompt: 1785
- Completion: 27
- Total: 1815

Total cost: $1.317



In [9]:
df["cards_label_pred"].value_counts()

cards_label_pred
0_accepted                 2673
4_solutions_wont_work        19
5_science_is_unreliable      17
3_its_not_bad                13
2_its_not_us                  9
1_its_not_happening           3
Name: count, dtype: int64

## Manually review the predictions with LabelStudio


In [2]:
df = pd.read_csv(
    "../../data/experiments/label_studio_review/sud_radio_itele_since_2024_07.csv",
    index_col=0,
)
df

,start,text,channel_name,channel_is_radio,channel_program_type,channel_program,themes,keywords,num_keywords,num_tokens,label_pred,prompt_tokens,completion_tokens,total_tokens,cost,claim_pred
id,,,,,,,,,,,,,,,,
c945dffa14ec20dd14b3e6e1304fbb4b25bddae8895e50d72d42e049e0ecee2e,2024-07-01 06:22:00,à la gestion de votre bien en location de la r...,itele,False,Information en continu,Information en continu,"[""changement_climatique_causes"", ""changement_c...","[{""keyword"": ""m\u00e9thane"", ""timestamp"": 1719...",1,506,0_accepted,1822,27,1849,0.000489,NaN
c90b7ed1e7139b209c7cd9ca6006f0fbfaf2de457c4f919aba6f4104056c5832,2024-07-01 07:12:00,xxl maison mobilier design et décoration de vo...,itele,False,Information en continu,Information en continu,"[""attenuation_climatique_solutions"", ""changeme...","[{""keyword"": ""transition \u00e9nerg\u00e9tique...",1,401,0_accepted,1664,27,1691,0.000450,NaN
d206d8c5c48efe0a621275c4bbe627c72b4de484339aa75d39cfb5a82aa7052f,2024-07-01 07:50:00,répercutée sur les prix la hausse des coûts de...,itele,False,Information en continu,Information en continu,"[""ressources_solutions"", ""attenuation_climatiq...","[{""keyword"": ""biogaz"", ""timestamp"": 1719813025...",2,556,0_accepted,1895,27,1922,0.000508,NaN
a8b1d5a95027030b142487be65621d16d928265ac18429a9ee356ccca2179ddf,2024-07-01 10:46:00,monde le prix du gaz en prenant en compte qu'i...,itele,False,Information en continu,Information en continu,"[""ressources_solutions"", ""attenuation_climatiq...","[{""keyword"": ""biogaz"", ""timestamp"": 1719823566...",1,498,0_accepted,1786,27,1813,0.000480,NaN
f055f93229e59c00a3872625b0c4e557938279b71944983ada8e8c0f8fca7a77,2024-07-01 11:44:00,est encore acté avec cette image que je trouve...,itele,False,Information en continu,Information en continu,"[""changement_climatique_constat"", ""changement_...","[{""keyword"": ""eaux"", ""timestamp"": 171982713101...",2,596,0_accepted,1881,27,1908,0.000504,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121c2c2780f373e0d03ad266a5b4c11e87873e353954051693a978db123e1a04,2024-09-12 07:06:00,est la période la plus à risque pour les inond...,sud-radio,True,Information - Magazine,Le Grand Matin,"[""adaptation_climatique_solutions"", ""changemen...","[{""keyword"": ""inondation"", ""timestamp"": 172611...",2,575,0_accepted,1902,27,1929,0.000509,NaN
10e4d16cad946340c67738bdaa487603d4635c6d2db978b63aff13916a49cfab,2024-09-12 08:00:00,solaire point fr votre expert en panneaux sola...,sud-radio,True,Information - Magazine,Le Grand Matin,"[""ressources_solutions"", ""changement_climatiqu...","[{""keyword"": ""panneaux solaires"", ""timestamp"":...",1,526,0_accepted,1788,27,1815,0.000481,NaN
74f1b824b772877e6e05908a308a8ae31146b652ff50793bbd7fc9d4deb5e29c,2024-09-12 09:00:00,foyer trois mille neuf cent quatre-vingt-dix e...,sud-radio,True,Information - Magazine,Le Grand Matin,"[""ressources_solutions"", ""changement_climatiqu...","[{""keyword"": ""panneaux solaires"", ""timestamp"":...",1,520,0_accepted,1829,27,1856,0.000491,NaN


In [3]:
# Generated code!
def approximate_match(text: str, claim: str, threshold=0.6) -> tuple[int, int] | None:
    # Preprocess text to remove casing and accents
    text = unidecode(text.lower())
    claim = claim.lower()

    # Split text and claim into words
    text_words = text.split()
    claim_words = claim.split()

    # Initialize variables to keep track of best match
    best_match_score = 0
    best_match_range = (None, None)

    # Slide a window of claim length over text to find the best match
    for i in range(len(text_words) - len(claim_words) + 1):
        # Extract a slice of words from the text to compare with claim
        window = text_words[i : i + len(claim_words)]

        # Calculate similarity ratio between the window and claim
        similarity = difflib.SequenceMatcher(None, window, claim_words).ratio()

        # Check if the similarity is above the threshold and better than the previous best
        if similarity > threshold and similarity > best_match_score:
            best_match_score = similarity
            best_match_range = (i, i + len(claim_words) - 1)

    # Convert word indices to character indices
    if best_match_range != (None, None):
        start_word_idx, end_word_idx = best_match_range
        start_char_idx = len(" ".join(text_words[:start_word_idx])) + (
            1 if start_word_idx > 0 else 0
        )
        end_char_idx = len(" ".join(text_words[: end_word_idx + 1]))
        return (start_char_idx, end_char_idx)
    else:
        return None

In [18]:
# Format for LabelStudio
# Only review flagged transcripts
data = []
for id, row in df[df["label_pred"] != "0_accepted"].iterrows():
    # Find the extracted claim in the original text
    text: str = row["text"]
    claim_pred: str = row["claim_pred"]
    try:
        # Try to see if we have a perfect match of claim in text
        start = text.lower().index(claim_pred.lower())
        end = start + len(claim_pred)
    except ValueError:
        # ValueError: substring not found
        matching_result = approximate_match(text, claim_pred, threshold=0.5)
        if matching_result:
            start, end = matching_result
        else:
            # The LLM "extracted" a claim that doesn't exist in the text...
            start, end = None, None

    # Pre-annotate the task with the model's prediction
    # We skip it if we couldn't match the extracted claim with the original text
    if start is not None:
        data.append(
            {
                "data": {
                    "id": id,
                    "start": str(row["start"]),
                    "text": text,
                    "channel_name": row["channel_name"],
                    "channel_program_type": row["channel_program_type"],
                    "channel_program": row["channel_program"],
                    "label_pred": row["label_pred"],
                    "claim_pred": claim_pred,
                },
                "predictions": [
                    {
                        "model_version": "claude-3-haiku",
                        "result": [
                            {
                                "value": {
                                    "start": start,
                                    "end": end,
                                    "text": text[start:end],
                                    "labels": [row["label_pred"]],
                                },
                                "from_name": "label",
                                "to_name": "text",
                                "type": "labels",
                            }
                        ],
                    }
                ],
            }
        )

In [19]:
# Save the LabelStudio tasks for import
with open(
    "../../data/experiments/label_studio_review/sud_radio_itele_since_2024_07_label_studio_review.json",
    "w",
) as f:
    json.dump(data, f)